# Prepare dataset

In this notebook, we will prepare the data for use later.

In [11]:
import pandas as pd
import numpy as np

## Load the dataset

Here we use a public dataset containing all company names and addresses in Japan.  You can download the dataset from [here](https://info.gbiz.go.jp/hojin/DownloadTop).

In [12]:
data = pd.read_csv('./resource/company_address.csv', dtype=str)


In [13]:
# convert NaN to empty string
data = data.fillna("")
data.sample(5)

,法人番号,法人名,郵便番号,g1,g2,g3,g4,rest
1274975,7010401065758,株式会社KMSアソシエイツ,1050014,東京都,港区,芝,2-5-15-210,
499732,2030001016412,株式会社ケイアンドエフ,3300074,埼玉県,さいたま市浦和区,北浦和,2-2-14,
729885,6040003005020,合同会社エヌケイエス商会,2740825,千葉県,船橋市,前原西,1-5-19,
4692546,4300001006359,株式会社ツツミ,8490937,佐賀県,佐賀市,鍋島,3-2-4,
131827,9430001016480,株式会社ミントハウス,0600042,北海道,札幌市中央区,大通西,5-8,


In [14]:
data.columns

Index(['法人番号', '法人名', '郵便番号', 'g1', 'g2', 'g3', 'g4', 'rest'], dtype='object')

In [15]:
fields_for_index = ['法人名', '郵便番号', 'g1', 'g2', 'g3', 'g4', 'rest']    
# try a small set
data.sample(5)[fields_for_index].values

array([['有限会社ヨコカワ', '2750002', '千葉県', '習志野市', '実籾', '4-3-35-302', ''],
       ['有限会社櫻井', '3760011', '群馬県', '桐生市', '相生町', '4-14-1', ''],
       ['有限会社ハウジングエステート', '1160012', '東京都', '荒川区', '東尾久', '2-24-17', ''],
       ['株式会社多津味', '9830011', '宮城県', '仙台市宮城野区', '栄', '2-24-11', ''],
       ['有限会社兵庫赤帽', '6512112', '兵庫県', '神戸市西区', '大津和', '3-3-10', '']],
      dtype=object)

Merge fields:

In [16]:
data["concat_name"] = data['法人名'] +  " " + data["g1"] + data["g2"] + data["g3"] + " " + data["g4"] + " " + data["rest"]
data["concat_name"].sample(5).values

array(['株式会社駒坂パートナーズ 東京都新宿区上落合 2-5-10 ',
       'スマート&会計合同会社 福岡県那珂川市中原 3-127-707 ', '東陽工業株式会社 大阪府大阪市平野区西脇 4-1-50 ',
       '株式会社五洋 東京都品川区東五反田 1-11-9 ', '合同会社スリーヴィレッジ 大阪府大阪市平野区背戸口 1-14-9 '],
      dtype=object)

Now we will take the columns we need, before saving

In [17]:
fields_to_save = ["法人名", "concat_name"]
data = data[fields_to_save]

Run below cell if you want to take a subset only.

In [9]:
# data = data.sample(1000)

At last, we will store the new dataframe for use later

In [18]:
data.to_csv("./features_full.csv", index=False)